In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=cd431e6fdcc258eb53887045dbfccb001ace15b44541ac8412b6cb7fce33daf1
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop_words


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=4c151c254c944de782464b5e1b4f756388084cd3d58829abaa3b0fc62fee0346
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [4]:
path_to_file = 'evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
text = text + text

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
text_as_int, text, len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких дум и простоты;\n                        Но так и быть - рукой пристрастной\n                        Прими собранье пестрых глав,\n                        Полусмешных, полупечальных,\n                        Простонародных, идеальных,\n                        Небрежный плод моих забав,\n                        Бессонниц, легких вдохновений,\n                        Незрелых и увядших лет,\n                        Ума холодных наблюде

### train and target

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [18]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 131)


In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(64, None)]         0           []                               
                                                                                                  
 embedding (Embedding)          (64, None, 256)      33536       ['input_1[0][0]']                
                                                                                                  
 gru (GRU)                      (64, None, 1024)     3938304     ['embedding[0][0]']              
                                                                                                  
 concatenate (Concatenate)      (64, None, 1280)     0           ['embedding[0][0]',              
                                                                  'gru[0][0]']                

In [21]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 0.00330867, -0.01787211, -0.02778577, ...,  0.01017553,
         0.00125837, -0.0353471 ],
       [ 0.00577582, -0.01668057, -0.01608414, ...,  0.02230806,
         0.00577065, -0.04457735],
       [ 0.00826718, -0.01531332, -0.00728233, ...,  0.0308793 ,
         0.00738096, -0.05001528],
       ...,
       [ 0.01203339, -0.014791  , -0.01453454, ...,  0.03604042,
         0.00188435, -0.05879902],
       [ 0.01277809, -0.01413355, -0.00490282, ...,  0.04100799,
         0.00442174, -0.05921366],
       [ 0.01333526, -0.01317825,  0.00090272, ...,  0.04402201,
         0.00586444, -0.05887017]], dtype=float32)>

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("-------------------------------------")
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '         Лишь только вдоль большой дороги\n                        Заслышат их домашни дроги, -\n     '
-------------------------------------
Next Char Predictions: 
 'аyпsDУК1!fh!!ЖяOAмNn-цСpMЯrз4nAЭхkИмИ2и(ЭЮщуУgъ8-джЗАмФ.:tsсшА6v5bDиРгVлIhhвOъfQЧ6Gr.)Q.ЮOdэпгzGэ}bК'


### Train the model

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8852277


In [25]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
!rm -rf "/content/drive/MyDrive/Colab Notebooks/training_checkpoints"

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/training_checkpoints"

ls: cannot access '/content/drive/MyDrive/Colab Notebooks/training_checkpoints': No such file or directory


In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [27]:
EPOCHS = 10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
88/88 [==============================] - 29s 283ms/step - loss: 1.9011
Epoch 2/10
88/88 [==============================] - 27s 294ms/step - loss: 1.3204
Epoch 3/10
88/88 [==============================] - 28s 302ms/step - loss: 1.2267
Epoch 4/10
88/88 [==============================] - 26s 285ms/step - loss: 1.1601
Epoch 5/10
88/88 [==============================] - 26s 289ms/step - loss: 1.0917
Epoch 6/10
88/88 [==============================] - 27s 301ms/step - loss: 1.0021
Epoch 7/10
88/88 [==============================] - 26s 289ms/step - loss: 0.9053
Epoch 8/10
88/88 [==============================] - 26s 287ms/step - loss: 0.7879
Epoch 9/10
88/88 [==============================] - 27s 295ms/step - loss: 0.6732
Epoch 10/10
88/88 [==============================] - 26s 291ms/step - loss: 0.5418


In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 131)


In [30]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [31]:
text_ = generate_text(model, start_string=u"Умом и страстью ")
print(text_)

Умом и страстью без собой,
                                                                                                                                                                                                                                                                                                                                                                                                                                                       Пред заветные расценал.
                          


In [32]:
text_ = generate_text(model, start_string=u"Умом и ")
print(text_)

Умом и стромах,
                        Другиим слушать шум работый,
                                                                                                                                                                                                                                                                                                                           С ней обратясь с волонами,
                                            Все, все, что в мало возок пробуке,
                
